# Text summarization with small files

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview

In this example, you are going to ingest a small amount of data (String data) directly into Amazon Bedrock API and give it an instruction to summarize the respective text.

### Architecture

![](../imgs/41-text-simple-1.png)

In this architecture:

1. A small piece of text (or small file) is loaded
1. A foundation model processes those data
1. Model returns a response with the summary of the ingested text

### Use case

This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Challenges
This approach can be used when the input text or file fits within the model context length. In notebook `06_OpenSource_examples/00_Langchain_TextGeneration_examples/04_long text summarization using LCEL chains on Langchain.ipynb`, we will explore an approach to address the challenge when users have large document(s) that exceed the token limit.

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️

In [2]:
import json
import os
import sys

import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

## Summarizing a short text with boto3
 
To learn detail of API request to Amazon Bedrock, this notebook introduces how to create API request and send the request via Boto3 rather than relying on langchain, which gives simpler API by wrapping Boto3 operation. 

### Request Syntax of InvokeModel in Boto3


We use `InvokeModel` API for sending request to a foundation model. 


### Writing prompt with text to be summarized

In this notebook, you can use any short text whose tokens are less than the maximum token of a foundation model. As an exmple of short text, let's take one paragraph of an [AWS blog post](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) about announcement of Amazon Bedrock.

The prompt starts with an instruction `Please provide a summary of the following text.`, and includes text surrounded by  `<text>` tag. 

In [3]:
prompt = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

"""

## Creating request body with prompt and inference parameters 

Following the request syntax of `invoke_model`, you create request body with the above prompt and inference parameters.

### Claude:

In [4]:
prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

In [5]:
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
max_tokens = 4096
top_p = 0.5
top_k = 250
temperature = 0.5

# Prompt with user turn only.
messages = [{"role": "user", "content":  prompt}]

In [6]:
body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages, 
            "temperature": temperature,
            "top_p": top_p, 
            "top_k": top_k
        }
    )

response = boto3_bedrock.invoke_model(body=body, modelId=model_id)
response_body = json.loads(response.get('body').read())
print(response_body['content'][0]['text'])

Here is a summary of the provided text:

AWS has announced a new service called Amazon Bedrock that provides access to foundation models (FMs) from AI companies like AI21 Labs, Anthropic, Stability AI, and Amazon itself via an API. Bedrock aims to democratize access to powerful generative AI models for text and images, including Amazon's own new Titan large language models (LLMs) that are being announced. 

Bedrock offers a serverless experience that allows customers to easily find the right model for their needs, get started quickly, customize models with their own data, and integrate them into applications using familiar AWS tools and services. This eliminates the need to manage infrastructure. Key features include the ability to privately customize FMs, test different models using Amazon SageMaker Experiments, and manage models at scale with SageMaker Pipelines.

Overall, Bedrock is positioned as the easiest way for AWS customers to build and scale applications using the latest gene

## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating a summary of AWS news about Amazon Bedrock.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.